# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846642905048                   -0.70    4.5         
  2   -7.852302271753       -2.25       -1.53    1.0   17.8ms
  3   -7.852608947795       -3.51       -2.56    1.5   19.2ms
  4   -7.852646050042       -4.43       -2.88    2.8   25.6ms
  5   -7.852646489518       -6.36       -3.14    1.0   17.7ms
  6   -7.852646680360       -6.72       -4.13    1.0   18.1ms
  7   -7.852646686684       -8.20       -5.00    2.0   22.7ms
  8   -7.852646686724      -10.40       -5.46    1.5   19.9ms
  9   -7.852646686727      -11.65       -5.48    1.5   20.6ms
 10   -7.852646686730      -11.50       -6.74    1.0   18.4ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846937148608                   -0.70           4.8         
  2   -7.852552439214       -2.25       -1.64   0.80    2.0    229ms
  3   -7.852638760920       -4.06       -2.74   0.80    1.0   16.9ms
  4   -7.852646454935       -5.11       -3.36   0.80    2.0   20.9ms
  5   -7.852646679810       -6.65       -4.23   0.80    1.8   19.9ms
  6   -7.852646686607       -8.17       -4.77   0.80    2.0   20.8ms
  7   -7.852646686723       -9.94       -5.68   0.80    1.2   18.3ms
  8   -7.852646686730      -11.16       -6.48   0.80    2.0   20.7ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.422031e+01     3.727097e+00
 * time: 0.047796010971069336
     1     1.494242e+00     1.758524e+00
 * time: 0.23592901229858398
     2    -1.372208e+00     2.020395e+00
 * time: 0.25429320335388184
     3    -3.505721e+00     1.721853e+00
 * time: 0.28093719482421875
     4    -4.554601e+00     1.753489e+00
 * time: 0.30747509002685547
     5    -6.452153e+00     1.284538e+00
 * time: 0.3340442180633545
     6    -7.311858e+00     8.396919e-01
 * time: 0.3609139919281006
     7    -7.601559e+00     9.160972e-01
 * time: 0.3795762062072754
     8    -7.698789e+00     7.048695e-01
 * time: 0.39809203147888184
     9    -7.743235e+00     5.639370e-01
 * time: 0.4163811206817627
    10    -7.771121e+00     1.813158e-01
 * time: 0.43457818031311035
    11    -7.789817e+00     1.163395e-01
 * time: 0.45302414894104004
    12    -7.817251e+00     1.298527e-01
 * time: 0.47123003005981445
    13    -7.832569e+00     1.063942e-01
 * time: 0.

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846958799816                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645941162                   -1.65         
  2   -7.852646686730       -6.13       -3.72    1.68s
  3   -7.852646686730      -13.36       -7.27    111ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.189411279597638e-7
|ρ_newton - ρ_scfv| = 9.185331926812569e-8
|ρ_newton - ρ_dm|   = 8.565602914283476e-10
